# Sparse Sinkhorn Transformer (PyTorch/GPU) (Ver 1.0)

***

### Credit for the PyTorch Reformer implementation goes out to @lucidrains of GitHub:

https://github.com/lucidrains/sinkhorn-transformer

***

This is a work in progress so please check back for updates.

***

Project Los Angeles

Tegridy Code 2021

In [ ]:
!git clone https://github.com/asigalov61/tegridy-tools
!pip install sinkhorn_transformer
!pip install local-attention

In [ ]:
%cd /content/tegridy-tools/tegridy-tools/
import TMIDI
%cd /content/

from sinkhorn_transformer import SinkhornTransformerLM
from sinkhorn_transformer.autoregressive_wrapper import AutoregressiveWrapper

import random
import tqdm
import gzip
import numpy as np
import torch
import torch.optim as optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset

# constants

NUM_BATCHES = int(1e5)
BATCH_SIZE = 28
GRADIENT_ACCUMULATE_EVERY = 4
LEARNING_RATE = 1e-4
VALIDATE_EVERY  = 100
GENERATE_EVERY  = 500
GENERATE_LENGTH = 512
SEQ_LEN = 4096

# helpers

def cycle(loader):
    while True:
        for data in loader:
            yield data

def decode_token(token):
    return str(chr(max(32, token)))

def decode_tokens(tokens):
    return ''.join(list(map(decode_token, tokens)))

# instantiate model

model = SinkhornTransformerLM(
    num_tokens = 256,
    emb_dim = 128,
    dim = 512,
    depth = 8,
    max_seq_len = SEQ_LEN,
    heads = 8,
    bucket_size = 128,
    ff_chunks = 2,
    causal = True,
    reversible = True,
    attn_dropout = 0.1,
    n_local_attn_heads = 4
)

model = AutoregressiveWrapper(model)
model.cuda()

# prepare enwik8 data

#@title Process the TXT MIDI dataset to TXT INT MIDI dataset

with open('/content/Optimus-VIRTUOSO-Music-Dataset.txt', 'r') as file:
  z = file.read()
  Z = z.encode('utf8')
  Y = list(Z)

trX, vaX = np.split(Y, [1000000])
data_train, data_val = torch.from_numpy(trX), torch.from_numpy(vaX)

class TextSamplerDataset(Dataset):
    def __init__(self, data, seq_len):
        super().__init__()
        self.data = data
        self.seq_len = seq_len

    def __getitem__(self, index):
        rand_start = torch.randint(0, self.data.size(0) - self.seq_len - 1, (1,))
        full_seq = self.data[rand_start: rand_start + self.seq_len + 1].long()
        return full_seq.cuda()

    def __len__(self):
        return self.data.size(0) // self.seq_len

train_dataset = TextSamplerDataset(data_train, SEQ_LEN)
val_dataset   = TextSamplerDataset(data_val, SEQ_LEN)
train_loader  = cycle(DataLoader(train_dataset, batch_size = BATCH_SIZE))
val_loader    = cycle(DataLoader(val_dataset, batch_size = BATCH_SIZE))

# optimizer

optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)


In [ ]:
# training

for i in tqdm.tqdm(range(NUM_BATCHES), mininterval=10., desc='training'):
    model.train()

    for __ in range(GRADIENT_ACCUMULATE_EVERY):
        loss = model(next(train_loader), return_loss = True)
        loss.backward()

    print(f'training loss: {loss.item()}')
    torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
    optim.step()
    optim.zero_grad()

    if i % VALIDATE_EVERY == 0:
        model.eval()
        with torch.no_grad():
            loss = model(next(val_loader), return_loss = True)
            print(f'validation loss: {loss.item()}')

    if i % GENERATE_EVERY == 0:
        model.eval()
        inp = random.choice(val_dataset)[:-1]
        prime = decode_tokens(inp)
        print(f'%s \n\n %s', (prime, '*' * 100))

        sample = model.generate(inp, GENERATE_LENGTH)
        output_str = decode_tokens(sample)
        print(output_str)

In [ ]:
torch.save(model.state_dict, '/content/model_sd.pth')
torch.save(model, '/content/model.pth')

In [ ]:
model.eval()
inp = random.choice(val_dataset)[:-1]
prime = decode_tokens(inp)
print(f'%s \n\n %s', (prime, '*' * 100))

sample = model.generate(inp, 2048)
output_str = decode_tokens(sample)
print(output_str)

In [ ]:
S = TMIDI.Tegridy_Optimus_TXT_to_Notes_Converter('SONG=TEST' + output_str, line_by_line_dataset=False, has_velocities=True, has_MIDI_channels=False, dataset_MIDI_events_time_denominator=10, char_encoding_offset=0, save_only_first_composition=True, simulate_velocity=False)
stats = TMIDI.Tegridy_SONG_to_MIDI_Converter(SONG=S[0], output_file_name='/content/test')
print(stats)